In [3]:
import re

def label_node(ckt_graph,input_vars,key):
    if key in input_vars:
        ckt_graph[key]["level"] = 0
        return 1
    elif ckt_graph[key]["level"] != -1:
        return ckt_graph[key]["level"]+1
    else:
        max_level = -1 
        for input in ckt_graph[key]["inputs"]:
            temp_level = label_node(ckt_graph,input_vars,input)
            if temp_level>max_level:
                max_level = temp_level
        ckt_graph[key]["level"] = max_level
        return max_level+1

def bench2graph(benchfile):
    with open(benchfile, 'r') as file:
        lines = file.readlines()
    
    ckt_graph={}
    output_vars=[]
    input_vars=[]
    wires = []
    input_pattern = re.compile(r'^INPUT\((\w+)\)')
    output_pattern = re.compile(r'^OUTPUT\((\w+)\)')
    
    for line in lines:
        line = line.strip()
        
        if "=" in line:
            gate_match = re.findall(r'\b\w+\b', line)
            var_name = gate_match[0].strip()
            wires.append(var_name)
            if var_name in ckt_graph.keys():
                ckt_graph[var_name]["gate"] = gate_match[1]
                ckt_graph[var_name]["inputs"] = gate_match[2:]
            else:  
                node={}  
                node["level"] = -1
                node["outbound"] = []
                node["gate"] = gate_match[1]
                node["inputs"] = gate_match[2:]
                ckt_graph[var_name] = node
            for inp_node in gate_match[2:]:
                if inp_node in ckt_graph.keys():
                    ckt_graph[inp_node]["outbound"].append(var_name)
                else:
                    node={}
                    node["level"] = -1
                    node["outbound"] = [var_name]
                    node["gate"] = ""
                    node["inputs"] = []
                    ckt_graph[inp_node] = node
                    
        elif len(line)>3:
            if line.startswith("INPUT"):
                match = input_pattern.match(line)
                if match:
                    input_vars.append(match.group(1))
            elif line.startswith("OUTPUT"):
                match = output_pattern.match(line)
                if match:
                    output_vars.append(match.group(1))
    
    for key in ckt_graph.keys():
        if key in output_vars:
            label_node(ckt_graph,input_vars,key)

    return ckt_graph,input_vars,output_vars,wires


def graph2bench(ckt_node,inputs,outputs,bench_name,input_node =False):
    bench=[f"# {bench_name}",f"# {len(inputs)} INPUTS", f"# {len(outputs)} OUTPUTS", f"# {len(ckt_node.keys())} Gates", "\n"]

    for input in inputs:
        bench.append(f"INPUT({input})")
    for output in outputs:
        bench.append(f"OUTPUT({output})")
    bench.append("\n")

    gate_no = 0
    key_len = len(ckt_node.keys())
    if input_node:
        key_len -= len(inputs)
    appending_level = 1
    while gate_no<key_len:
        for key in ckt_node.keys():
            if ckt_node[key]["level"]==appending_level:
                if len(ckt_node[key]["inputs"])>1:
                    gate_inp = ", ".join(ckt_node[key]["inputs"])
                else: gate_inp = ckt_node[key]["inputs"][0]
                gate_name = ckt_node[key]["gate"]
                bench.append(f"{key} = {gate_name}({gate_inp})")
                gate_no += 1
        appending_level += 1

    with open(bench_name+".bench", 'w') as file:
        file.write("\n".join(bench))
        print(bench_name+" bench file created")
                

In [2]:
import re
def nandlib_synth2bench(file):
    with open(file, 'r') as file_name:
        file_content = file_name.read()
    file_content = file_content.replace("\n","")
    file_content = file_content.split(";")

    gate_pattern = r'\b(AND|OR|NAND|NOR|XOR|XNOR|INV)'
    pin_pattern = r'\(\s*([^\(\)]+?)\s*\)'
    gates=[]
    for line in file_content:
        line = line.strip()
        gate = re.findall(gate_pattern, line)
        if gate:
            if gate[0] == "INV": gate[0] = "NOT"
            line = line.replace("K[","keyinput")
            line = line.replace("]","")
            line = line.replace("PPI[","in_")
            line = line.split(".Z")
            in_pin = re.findall(pin_pattern, line[0])
            out_pin = re.findall(pin_pattern, line[1])
            if len(in_pin)>1:
                inpin = ", ".join(in_pin)
            else:
                inpin = in_pin[0]
            gates.append(f"{out_pin[0]} = {gate[0]}({inpin})")

    
    return gates

In [ ]:
file = "D:\\Personal\\New folder\\BUET_SAT_SIMULATOR\\src\\cac_synth16.v"
gates = nandlib_synth2bench(file)

inp=["INPUT(OPO)","OUTPUT(LPO)"]
key=[]
for i in range(16):
    inp.append(f"INPUT(in_{i})")
    key.append(f"INPUT(keyinput{i})")

file = file.replace(".v",".bench")
with open(file, 'w') as file_name:
    file_name.write("\n".join(inp+key+gates))
    print(" bench file created")

ckt_graph,input_vars,output_vars,wires = bench2graph(file)
graph2bench(ckt_graph,input_vars,output_vars,"cac",True)

In [12]:

def convert_gate_line(line, gate_count):
    match = re.findall(r'\b\w+\b', line)
    gate_dict=[ "AND", "NAND", "OR", "NOR", "XOR", "XNOR", "NOT", "BUF", "DFF"]
    
    if match:
        output_var = match[0]
        gate_type = match[1]  # Convert gate type to lowercase
        inputs = match[2:]
        
        if gate_type in gate_dict:
            gate_unique_name = f"{gate_type}{len(inputs)}_{gate_count}"
            converted_line = f"{gate_type.lower()} {gate_unique_name} ({output_var}, {', '.join(inputs)});"
            return gate_type, converted_line
    
    return None, None


def bench2verilog(file_path, file_name):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    gate_count = 1
    input_vars = []
    output_vars = []
    assigned_vars = []
    dff_module = [
            "module DFF(Q, clk, D);",
            "input D;",
            "input clk;",
            "output Q;",
            "always @(clk)", 
            "begin",
            "  Q <= D;", 
            "end", 
            "endmodule"]
    opr_lines=[]
    gate_types=[]

    # Regular expressions to match INPUT, OUTPUT, and gate assignments
    input_pattern = re.compile(r'^INPUT\((\w+)\)')
    output_pattern = re.compile(r'^OUTPUT\((\w+)\)')
    assign_pattern = re.compile(r'^(\w+)\s*=\s*(NAND|NOR|AND|OR|XOR|XNOR|NOT|DFF|BUF)\((.*)\)')

    for line in lines:
        line = line.strip()
        if line.startswith("INPUT"):
            match = input_pattern.match(line)
            if match:
                input_vars.append(match.group(1))
        elif line.startswith("OUTPUT"):
            match = output_pattern.match(line)
            if match:
                output_vars.append(match.group(1))
        else:
            match = assign_pattern.match(line)
            if match:
                assigned_vars.append(match.group(1))
                gate_type,opr_line = convert_gate_line(line,gate_count)
                if opr_line is not None:
                    opr_lines.append(opr_line)
                    if gate_type not in gate_types:
                        gate_types.append(gate_type) 
                    gate_count += 1

    # Intermediate variables are those assigned but not inputs or outputs
    intermediate_vars = [var for var in assigned_vars if var not in input_vars and var not in output_vars]

    module_heading = f"module {file_name}({','.join(input_vars + output_vars)});"
    input_line = f"input {','.join(input_vars)};"
    output_line = f"output {','.join(output_vars)};"
    wire_line = f"wire {','.join(intermediate_vars)};"
    termination_line = "endmodule"
    main_module="\n".join([module_heading,"",input_line,output_line,wire_line,""]+opr_lines+["",termination_line])
    
    if "DFF" in gate_types:
        main_module = "\n".join(dff_module + ["\n", main_module])

    with open(file_name+".v", 'w') as file:
        file.write(main_module)
        print(file_name+" verilog file created")


In [28]:
def build_cone_backward(cone_graph,ckt_graph,inputs,key):
    for input in ckt_graph[key]["inputs"]:
        cone_graph[key] = ckt_graph [key]
        if input not in cone_graph.keys():
            build_cone_backward(cone_graph,ckt_graph,inputs,input) 
    return

def build_cone_forward(cone_graph,ckt_graph,inputs,outputs,cone_output,key,lvl):
    if key in outputs:
        cone_graph[key] = ckt_graph [key]
        cone_output.append(key)
        return
    else:
        for outbound in ckt_graph[key]["outbound"]:
            if outbound not in cone_graph.keys():
                cone_graph[outbound] = ckt_graph [outbound]
                build_cone_forward(cone_graph,ckt_graph,inputs,outputs,cone_output,outbound,lvl+1)
        if lvl<4:
            for input in ckt_graph[key]["inputs"]:
                if input not in cone_graph.keys():
                    build_cone_backward(cone_graph,ckt_graph,inputs,input)
            
        return


def logic_cone_forward(benchfile,target_text,cone_name):
    ckt_graph,inputs,outputs,wires=bench2graph(benchfile)
    cone_graph={}
    cone_input=[]
    cone_output=[]
    for input in inputs:
        if target_text in input:
            cone_input.append(input)
            build_cone_forward(cone_graph,ckt_graph,inputs,outputs,cone_output,input,1)
    

    cone_wires = list(cone_graph.keys())
    inpx=[]
    wirex=[]
    for key in cone_graph.keys():
        new_inp=[]
        for input in cone_graph[key]["inputs"]:
            if target_text not in input:
                if input in inputs:
                    if input in inpx:
                        new_inp.append(f"INPUTx{inpx.index(input)}")
                    else:
                        new_inp.append(f"INPUTx{len(inpx)}")
                        inpx.append(input)
                elif input in cone_wires:
                    new_inp.append(input)
                elif input in wires:
                    if input in wirex:
                        new_inp.append(f"WIREx{wirex.index(input)}")
                    else:
                        new_inp.append(f"WIREx{len(wirex)}")
                        wirex.append(input)
                else:
                    print("undefined input") 
                    new_inp.append(input)
            else: new_inp.append(input)
        cone_graph[key]["inputs"] = new_inp


        
    graph2bench(cone_graph,cone_input,cone_output,cone_name)




In [ ]:
logic_cone_forward("/Users/mahmudulhasan/Downloads/BUET_SAT_SIMULATOR/c6288/c6288_ttlock_32k.bench","keyinput","ttlock")

In [ ]:
logic_cone_forward("/Users/mahmudulhasan/Downloads/BUET_SAT_SIMULATOR/bench_ckt/c17.bench","gatx","c17")

In [ ]:
bench2verilog("/home/mahmudul-hasan/Research/BUET_SAT_SIMULATOR/src/cac.bench","cac")

In [ ]:
#ttlock/cac injector
import random
ckt_name = "c1355"
bit_no = "8"
org_file = f"D:\\Personal\\New folder\\BUET_SAT_SIMULATOR\\bench_ckt\\{ckt_name}.bench"
lock_file = f"D:\\Personal\\New folder\\BUET_SAT_SIMULATOR\\obfuscated\\cac{bit_no}.bench"
file_name = f"D:\\Personal\\New folder\\BUET_SAT_SIMULATOR\\8bit\\{ckt_name}_cac_8k.bench"

def bench2list(org_file, add_out=True):
    with open(org_file, 'r') as file:
            lines = file.readlines()
        
    output_vars=[]
    input_vars=[]
    io_lines = []
    gate_lines = []
    wires = []
    input_pattern = re.compile(r'^INPUT\((\w+)\)')
    output_pattern = re.compile(r'^OUTPUT\((\w+)\)')
    
    for line in lines:
        line = line.strip()
        
        if "=" in line:
            gate_lines.append(line)
            gate_match = re.findall(r'\b\w+\b', line)
            var_name = gate_match[0].strip()
            wires.append(var_name)

        elif len(line)>3:
            if line.startswith("INPUT"):
                match = input_pattern.match(line)
                if match:
                    input_vars.append(match.group(1))
                    io_lines.append(line)
            elif line.startswith("OUTPUT"):
                match = output_pattern.match(line)
                if match:
                    if add_out:
                        io_lines.append(line)
                    output_vars.append(match.group(1))
                    
    
    return gate_lines,io_lines,input_vars,output_vars,wires

org_gates, org_io, org_inputs, org_outputs, org_wires = bench2list(org_file)
lock_gates, lock_io, lock_inputs, lock_outputs, lock_wires = bench2list(lock_file, False)

bench = org_io
for i in range(32):
    bench.append(f"INPUT(keyinput{i})")

gate_index = org_wires.index(org_outputs[-1])
org_gates[gate_index] = org_gates[gate_index].replace(org_outputs[-1],org_outputs[-1]+"_enc")
lock_gates[-1] = lock_gates[-1].replace(lock_outputs[0],org_outputs[-1])
lock_gates[-3] = lock_gates[-3].replace("OPO",org_outputs[-1]+"_enc")

wirex=[]
for i in range(len(lock_gates)):
    if "in_" in lock_gates[i]:
        gate_match = re.findall(r'\b\w+\b', lock_gates[i])
        for inp in gate_match[2:]:
            if "in_" in inp:
                inp_index = int(inp.split("_")[-1].strip())
                lock_gates[i] = lock_gates[i].replace(inp,org_inputs[inp_index])
    

bench += org_gates+lock_gates
with open(file_name, 'w') as file:
    file.write("\n".join(bench))
    print(file_name+" bench file created")

In [ ]:
import algo_methods

In [ ]:
org_ckt_ls = "/home/mahmudul-hasan/Research/BUET_SAT_SIMULATOR/c432/c432.bench"
key_str = "10011000111110101001100011111010"

In [ ]:
obs_ckt_ls = "/Users/mahmudulhasan/Downloads/BUET_SAT_SIMULATOR/c432/c432_sarlock_32k.bench"
algo_methods.sarlock(org_ckt_ls,obs_ckt_ls,key_str)

In [ ]:
obs_ckt_ls = "/Users/mahmudulhasan/Downloads/BUET_SAT_SIMULATOR/c6288/c6288_rll_32k.bench"
algo_methods.RLL(org_ckt_ls,obs_ckt_ls,key_str)

In [ ]:
obs_ckt_ls = "src/c6288_antisat_32k.bench"
algo_methods.libar(org_ckt_ls,obs_ckt_ls,key_str,libar_percent=0.6)

In [ ]:
obs_ckt_ls = "/Users/mahmudulhasan/Downloads/BUET_SAT_SIMULATOR/c432/c432_antisat_32k.bench"
algo_methods.anti_sat(org_ckt_ls,obs_ckt_ls,key_str)

In [ ]:
obs_ckt_ls = "src/c6288_antisat_32k.bench"
algo_methods.asob(org_ckt_ls,obs_ckt_ls,key_str,no_rll_keybit=6)

In [ ]:
org_ckt_ls = "/home/mahmudul-hasan/Research/BUET_SAT_SIMULATOR/bench_ckt/c6288.bench"
obs_ckt_ls = "/home/mahmudul-hasan/Research/BUET_SAT_SIMULATOR/8bit/c6288_rll.bench"

In [ ]:
org_ckt_ls = "/home/mahmudul-hasan/Research/BUET_SAT_SIMULATOR/c432/c432.bench"
obs_ckt_ls = "/home/mahmudul-hasan/Research/BUET_SAT_SIMULATOR/c432/c432_ttlock_32k.bench"

In [ ]:
algo_methods.sat(org_ckt_ls,obs_ckt_ls)

In [ ]:
algo_methods.appsat(org_ckt_ls,obs_ckt_ls)

In [ ]:
algo_methods.hamming_sweep(org_ckt_ls,obs_ckt_ls,max_iter=1000)

In [23]:
import algo_methods
import math
key8 = "10011001"
key16 = "1110100011101000"
key32 = "10011000111110101001100011111010"

key5 =  "10011"
key10 = "1110100010"
key15 = "100110001111101"
key20 = "10011000111110101001"



ckt = "c432"
bit_no = 5
key = key5
abs_path = "/Users/mahmudulhasan/Documents"


src = f"{abs_path}/BUET_SAT_SIMULATOR/bench_ckt/{ckt}.bench"
#rll= f"{abs_path}/BUET_SAT_SIMULATOR/{bit_no}bit/{ckt}_rll_{bit_no}k.bench"
rll= f"{abs_path}/BUET_SAT_SIMULATOR/rll_libar/{ckt}_rll_{bit_no}k.bench"




In [ ]:
algo_methods.RLL(src,rll,key)
algo_methods.sat(src,rll)

In [ ]:
ckt = "c1355"
rll= f"{abs_path}/BUET_SAT_SIMULATOR/rll_libar/{ckt}_rll_{bit_no}k.bench"

ovlp_pins =[0,1,2] 
libar_prcnt=[2,5,10,15,25,50,75,100]
keys = ["10011","1110100010","100110001111101","10011000111110101001"]

for key in keys:
    bit_no = len(key) 
    for ovlp_pin in ovlp_pins:
        for prcnt in libar_prcnt:
            libar_no = math.ceil(prcnt*int(bit_no)/100)
            algo_methods.libar(rll,f"{abs_path}/BUET_SAT_SIMULATOR/libars/{ckt}_{bit_no}k_{ovlp_pin}pinovlp_libar_{prcnt}.bench",key,libar_no,True,ovlp_pin)

: 

In [29]:
import os 
directory = os.path.join(abs_path,f"BUET_SAT_SIMULATOR/libars")
for dirpath, _, filenames in os.walk(directory):
    for filename in filenames:
        if "unrolled.bench" not in filename: 
            file_path = os.path.abspath(os.path.join(dirpath, filename))
            os.remove(file_path)
        

In [1]:
import csv


# Function to process each line
def process_line(line):
    parts = line.split(' ', 1)  # Split into circuit name and the rest
    circuit_name = parts[0]
    new_parts = parts[1].split(':') # Split into attack method and the rest
    attack = (new_parts[0].split(" ")[0]).strip()
    result_part = new_parts[1].strip()

    # Initialize default values
    key_found = "No"
    iteration = "-"
    time = "-"

    #extract value from circuit name
    c1_row = circuit_name.split("_")
    ckt = c1_row[0].strip()
    key_bit = c1_row[1].strip()
    clk_inp_olp = c1_row[2][0]
    libar_no = c1_row[4].strip()
    # Check if a key was found
    if "func_iteration" in result_part:
        if "key" in result_part:
            key = (result_part.split('key= ')[1].split(';')[0]).strip()
            keys_dict = {5:"10011",10:"1110100010",15:"100110001111101",20:"10011000111110101001"}
            org_key = keys_dict[len(key)]
            if key == org_key:
                key_found = "Yes"
                iteration = result_part.split('func_iteration= ')[1].split(';')[0]
                time = result_part.split('func_exe_time= ')[1].split(';')[0]

            
        else:
            key_found = "Yes"
            iteration = result_part.split('func_iteration= ')[1].split(';')[0]
            time = result_part.split('func_exe_time= ')[1].split(';')[0]
        
    
    return [ckt,key_bit,clk_inp_olp,libar_no, attack, key_found, iteration, time]


txt_file = "/home/mahmudul-hasan/Research/BUET_SAT_SIMULATOR/src/raw_rslt.txt"
csv_file = "/home/mahmudul-hasan/Research/BUET_SAT_SIMULATOR/src/result_libar.csv"

with open(txt_file, 'r') as file:
    lines = file.readlines()
# Process all lines
processed_lines = []
for line in lines:
    line = line.strip()
    if len(line)>10:
        processed_lines.append(process_line(line))



# Write or append to the result.csv file
with open(csv_file, mode='a', newline='') as file:
    writer = csv.writer(file)
    for row in processed_lines:
        writer.writerow(row)

print("Data appended to result.csv successfully!")


Data appended to result.csv successfully!


In [12]:
output_file ="D:\\Personal\\New folder\\BUET_SAT_SIMULATOR\\src\\libar_test.csv"
input_file = "D:\\Personal\\New folder\\BUET_SAT_SIMULATOR\\src\\result_libar.csv"
with open(input_file, mode='r', newline='') as infile:
    reader = csv.reader(infile)
    data = list(reader)  # Convert the reader object to a list

new_csv = [["ckt","key_bit","clk_inp_olp","libar_no","attack_method","key" "found","iteration","time"]]
for row in data:
    if len(row) > 2:
        c1_row = row[0].split("_")
        if len(c1_row)== 5:
            new_csv.append([c1_row[0].strip(),c1_row[1].strip(),c1_row[2][0],c1_row[4].strip()]+row[1:])

# Write modified data to the output CSV file
with open(output_file, mode='a', newline='') as outfile:
    writer = csv.writer(outfile)
    writer.writerows(new_csv)